```{autolink-concat}
```

::::{margin}
:::{card} Amplitude building with K-matrix dynamics
TR-031
^^^
Illustration of how to formulate an amplitude model with P-vector dynamics.
:::
::::

# P-vector fit comparison

In [ ]:
%pip install -q 'qrules[viz]==0.10.2' 'tensorwaves[jax,phsp]==0.4.12' ampform==0.15.4 pandas==2.2.2 sympy==1.12

In [ ]:
from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass
from typing import Any, Iterable, Mapping

import ampform
import attrs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import qrules
import sympy as sp
from ampform.dynamics.builder import TwoBodyKinematicVariableSet
from ampform.helicity import HelicityModel, ParameterValues
from ampform.io import aslatex
from ampform.kinematics.phasespace import Kallen
from ampform.sympy import perform_cached_doit, unevaluated
from IPython.display import Math, display
from matplotlib import cm
from qrules.particle import Particle, ParticleCollection
from qrules.transition import ReactionInfo
from sympy import Abs
from sympy.matrices.expressions.matexpr import MatrixElement
from tensorwaves.data import (
    IntensityDistributionGenerator,
    SympyDataTransformer,
    TFPhaseSpaceGenerator,
    TFUniformRealNumberGenerator,
    TFWeightedPhaseSpaceGenerator,
)
from tensorwaves.estimator import UnbinnedNLL
from tensorwaves.function.sympy import create_parametrized_function
from tensorwaves.interface import DataSample, Estimator, Function, ParameterValue
from tensorwaves.optimizer import Minuit2
from tensorwaves.optimizer.callbacks import CSVSummary

_ = np.seterr(invalid="ignore")

## Collect dynamics symbols

In [ ]:
def create_dynamics_symbol(
    resonance: Particle, variable_pool: TwoBodyKinematicVariableSet
) -> tuple[sp.Expr, dict[sp.Symbol, float]]:
    J = sp.Rational(resonance.spin)
    Q = resonance.charge
    P = sp.Rational(resonance.parity)
    if variable_pool.angular_momentum is not None:
        L = sp.Rational(variable_pool.angular_momentum)
        X = sp.Symbol(Rf"X_{{Q={Q:+d}, S={J}, P ={P}}}^{{l={L}}}")
    else:
        X = sp.Symbol(Rf"X_{{Q={Q:+d}, S={J}, P ={P}}}")
    COLLECTED_X_SYMBOLS[X].add((resonance, variable_pool))
    parameter_defaults = {}
    return X, parameter_defaults


COLLECTED_X_SYMBOLS = defaultdict(set)

In [ ]:
def load_particle_database() -> ParticleCollection:
    particle_database = qrules.load_default_particles()
    additional_definitions = qrules.io.load("030/additional-definitions.yml")
    particle_database.update(additional_definitions)
    return particle_database


PARTICLE_DB = load_particle_database()

In [ ]:
FINAL_STATES: list[tuple[str, ...]] = [
    ["K0", "Sigma+", "p~"],
    ["eta", "p", "p~"],
]
REACTIONS: list[ReactionInfo] = [
    qrules.generate_transitions(
        initial_state="J/psi(1S)",
        final_state=final_state,
        allowed_intermediate_particles=["N**1"],
        allowed_interaction_types=["strong"],
        formalism="helicity",
        particle_db=PARTICLE_DB,
    )
    for final_state in FINAL_STATES
]

In [ ]:
MODELS: list[HelicityModel] = []
for reaction in REACTIONS:
    builder = ampform.get_builder(reaction)
    builder.adapter.permutate_registered_topologies()
    builder.scalar_initial_state_mass = True
    builder.stable_final_state_ids = [0, 1, 2]
    for resonance in reaction.get_intermediate_particles():
        builder.set_dynamics(resonance.name, create_dynamics_symbol)
    MODELS.append(builder.formulate())

In [ ]:
selected_amplitudes = {
    k: v for i, (k, v) in enumerate(MODELS[0].amplitudes.items()) if i < 3
}

In [ ]:
for X, resonance_info in COLLECTED_X_SYMBOLS.items():
    for res, _ in sorted(resonance_info):
        display(X)
        print(f"  {res.name:<20s} {res.mass:>8g} GeV  {res.width:>8g} GeV")

In [ ]:
@dataclass
class TwoBodyDecay:  # specific to the channel
    child1: Particle
    child2: Particle


DECAYS = tuple(
    TwoBodyDecay(
        child1=reaction.final_state[0],
        child2=reaction.final_state[1],
    )
    for reaction in REACTIONS
)
s = sp.Symbol("m_01", real=True) ** 2

PARAMETERS_DEFAULTS = {}
for model in MODELS:
    PARAMETERS_DEFAULTS.update(model.parameter_defaults)
    del model

resonances, *_ = COLLECTED_X_SYMBOLS.values()

## Formulate dynamics expression

In [ ]:
for symbol, resonances in COLLECTED_X_SYMBOLS.items():
    display(symbol)
    for p, _ in resonances:
        print(f"  {p.name:<20s} {p.mass:>8g} GeV  {p.width:>8g} GeV ")
MODELS[0].parameter_defaults

## Formulate Dynamics

### Phasespace factor

In [ ]:
@unevaluated(real=False)
class PhaseSpaceCM(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho^\mathrm{{CM}}_{{{m1},{m2}}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return -16 * sp.pi * sp.I * ChewMandelstam(s, m1, m2)


@unevaluated(real=False)
class ChewMandelstam(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\Sigma\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        q = BreakupMomentum(s, m1, m2)
        return (
            1
            / (16 * sp.pi**2)
            * (
                (2 * q / sp.sqrt(s))
                * sp.log(Abs((m1**2 + m2**2 - s + 2 * sp.sqrt(s) * q) / (2 * m1 * m2)))
                - (m1**2 - m2**2) * (1 / s - 1 / (m1 + m2) ** 2) * sp.log(m1 / m2)
            )
        )


@unevaluated(real=False)
class BreakupMomentum(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"q\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt(Kallen(s, m1**2, m2**2)) / (2 * sp.sqrt(s))


s, m1, m2 = sp.symbols("s m1 m2")
exprs = [
    PhaseSpaceCM(s, m1, m2),
    ChewMandelstam(s, m1, m2),
    BreakupMomentum(s, m1, m2),
]
Math(aslatex({e: e.doit(deep=False) for e in exprs}))

In [ ]:
@unevaluated(real=False)
class ChannelWidth(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    width: Any
    _latex_repr_ = R"\Gamma_s\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2, width = self.args
        return width * PhaseSpaceCM(s, m1, m2)


s, m1, m2, width = sp.symbols("s m1 m2 gamma0")
expr = ChannelWidth(s, m1, m2, width)
Math(aslatex({expr: expr.doit(deep=False)}))

In [ ]:
@unevaluated(real=False)
class CM(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"CM_{{{m1},{m2}}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return (
            -2
            / sp.pi
            * (
                -1
                / s
                * sp.sqrt(((m1 + m2) ** 2 - s) * ((m1 - m2) ** 2 - s))
                * sp.log(
                    (sp.sqrt((m1 + m2) ** 2 - s) + sp.sqrt((m1 - m2) ** 2 - s))
                    / (2 * sp.sqrt(m1 * m2))
                )
                + (m1**2 - m2**2) / (2 * s) * sp.log(m1 / m2)
                - (m1**2 + m2**2) / (2 * (m1**2 - m2**2)) * sp.log(m1 / m2)
                - 1 / 2
            )
        )


s, m1, m2 = sp.symbols("s m1 m2")
CM_expr = CM(s, m1, m2)
Math(aslatex({CM_expr: CM_expr.doit(deep=False)}))

### 

Relativistic Breit-Wigner

In [ ]:
PARAMETERS_BW = {}


def formulate_breit_wigner(
    resonances: list[tuple[Particle, TwoBodyKinematicVariableSet]],
) -> sp.Expr:
    (_, variables), *_ = resonances
    s = variables.incoming_state_mass**2
    m_a = variables.outgoing_state_mass1
    m_b = variables.outgoing_state_mass2
    w = [sp.Symbol(Rf"w_{{{p.latex}}}") for p, _ in resonances]
    m = [sp.Symbol(Rf"m_{{{p.latex}}}") for p, _ in resonances]
    b = [sp.Symbol(Rf"b_{{{p.latex}}}") for p, _ in resonances]
    d = [sp.Symbol(Rf"d_{{{p.latex}}}") for p, _ in resonances]
    L = [sp.Symbol(Rf"L_{{{p.latex}}}") for p, _ in resonances]
    w_s = (ChannelWidth(s, m_a, m_b, w_) for w_ in w)
    rel_bw = sum((w_ * m_) / (m_**2 - s - m_ * w_s_) for m_, w_, w_s_ in zip(m, w, w_s))
    for i, (resonance, _) in enumerate(resonances):
        PARAMETERS_BW[w[i]] = resonance.width
        PARAMETERS_BW[m[i]] = resonance.mass
        PARAMETERS_BW[b[i]] = 1
        PARAMETERS_BW[d[i]] = 1
        PARAMETERS_BW[L[i]] = 0
    return rel_bw

### Define matrix symbols

In [ ]:
n_channels = len(REACTIONS)
I = sp.Identity(n_channels)
K = sp.MatrixSymbol("K", n_channels, n_channels)
P = sp.MatrixSymbol("P", n_channels, 1)
F = sp.MatrixSymbol("F", n_channels, 1)
rho = sp.MatrixSymbol("rho", n_channels, n_channels)



### $K$ matrix 

In [ ]:
def formulate_k_matrix(
    resonances: list[tuple[Particle, int]], n_channels: int
) -> dict[MatrixElement, sp.Expr]:
    Kmatrix_expressions = {}
    for i in range(n_channels):
        for j in range(n_channels):
            resonance_contributions = []
            for res, _ in resonances:
                s = sp.Symbol("m_01", real=True) ** 2
                m_a_i = sp.Symbol(Rf"m_{{0,{i}}}")
                m_b_i = sp.Symbol(Rf"m_{{1,{i}}}")
                m_a_j = sp.Symbol(Rf"m_{{0,{j}}}")
                m_b_j = sp.Symbol(Rf"m_{{1,{j}}}")
                g_Ri = sp.Symbol(Rf"g_{{{res.latex},{i}}}")
                g_Rj = sp.Symbol(Rf"g_{{{res.latex},{j}}}")
                m_R = sp.Symbol(Rf"m_{{{res.latex}}}")
                parameter_defaults = {
                    m_a_i: DECAYS[i].child1.mass,
                    m_b_i: DECAYS[i].child2.mass,
                    m_a_j: DECAYS[j].child1.mass,
                    m_b_j: DECAYS[j].child2.mass,
                    m_R: res.mass,
                    g_Ri: 1,
                    g_Rj: 0.1,
                }
                PARAMETERS_DEFAULTS.update(parameter_defaults)
                expr = (g_Ri * g_Rj) / (m_R**2 - s)
                resonance_contributions.append(expr)
            Kmatrix_expressions[K[i, j]] = sum(resonance_contributions)

    return Kmatrix_expressions


K_expressions = formulate_k_matrix(resonances, n_channels=len(REACTIONS))
Math(aslatex(K_expressions))
K_matrix = K.as_explicit()
K.as_explicit().xreplace(K_expressions)

### $P$ vector

In [ ]:
def formulate_p_vector(
    resonances: list[tuple[Particle, int]], n_channels: int
) -> dict[MatrixElement, sp.Expr]:
    P_expressions = {}
    for i in range(n_channels):
        resonance_contributions = []
        for res, _ in resonances:
            s = sp.Symbol("m_01", real=True) ** 2
            m_a = sp.Symbol(Rf"m_{{0,{i}}}")
            m_b = sp.Symbol(Rf"m_{{1,{i}}}")
            g_Ri = sp.Symbol(Rf"g_{{{res.latex},{i}}}")
            beta_R = sp.Symbol(Rf"\beta_{{{res.latex}}}")
            m_R = sp.Symbol(Rf"m_{{{res.latex}}}")

            parameter_defaults = {
                m_a: DECAYS[i].child1.mass,
                m_b: DECAYS[i].child2.mass,
                m_R: res.mass,
                beta_R: 1 + 0j,
                g_Ri: 1,
            }
            PARAMETERS_DEFAULTS.update(parameter_defaults)
            expr = (beta_R * g_Ri) / (m_R**2 - s)
            resonance_contributions.append(expr)
        P_expressions[P[i, 0]] = sum(resonance_contributions)

    return P_expressions


P_expressions = formulate_p_vector(resonances, n_channels=len(REACTIONS))
Math(aslatex(P_expressions))
P_vector = P.as_explicit()
P.as_explicit().xreplace(P_expressions)

### Phase space

In [ ]:
def formulate_phsp_factor_matrix(n_channels: int) -> dict[sp.MatrixElement, sp.Expr]:
    matrix_expressions = {}

    for i in range(n_channels):
        for j in range(n_channels):
            if i == j:
                m_a_i = sp.Symbol(Rf"m_{{0,{i}}}")
                m_b_i = sp.Symbol(Rf"m_{{1,{i}}}")
                s = sp.Symbol("m_01", real=True) ** 2
                rho_i = PhaseSpaceCM(s, m_a_i, m_b_i)
                matrix_expressions[rho[i, j]] = rho_i
                parameter_defaults = {
                    m_a_i: DECAYS[i].child1.mass,
                    m_b_i: DECAYS[i].child2.mass,
                }
                PARAMETERS_DEFAULTS.update(parameter_defaults)
            else:
                matrix_expressions[rho[i, j]] = 0

    return matrix_expressions


rho_expressions = formulate_phsp_factor_matrix(n_channels=len(REACTIONS))
rho.as_explicit().xreplace(rho_expressions)

### $F$ vector

:::{note}
For some reason one has to leave out the multiplication of $\rho$ by $i$ within the calculation of the $F$ vector
:::

In [ ]:
F = (I - sp.I * K * rho).inv() * P
F

In [ ]:
F_vector = F.as_explicit()

In [ ]:
combined_expressions = {**K_expressions, **rho_expressions, **P_expressions}
F_exprs = F_vector.xreplace(combined_expressions)
F_exprs[0].simplify(doit=False)

### Model $F$ vector

In [ ]:
F_unfolded_exprs = np.array([perform_cached_doit(expr) for expr in F_exprs])

In [ ]:
DYNAMICS_EXPRESSIONS_FVECTOR = []
for i in range(n_channels):
    exprs = {
        symbol: F_unfolded_exprs[i]
        for symbol, resonances in COLLECTED_X_SYMBOLS.items()
    }
    DYNAMICS_EXPRESSIONS_FVECTOR.append(exprs)

MODELS_FVECTOR = []
for i in range(n_channels):
    MODELS_FVECTOR.append(
        attrs.evolve(
            MODELS[i],
            parameter_defaults=ParameterValues({
                **MODELS[i].parameter_defaults,
                **PARAMETERS_DEFAULTS,
            }),
        )
    )

In [ ]:
FULL_EXPRESSIONS_FVECTOR = []
for i in range(n_channels):
    FULL_EXPRESSIONS_FVECTOR.append(
        perform_cached_doit(MODELS_FVECTOR[i].expression).xreplace(
            DYNAMICS_EXPRESSIONS_FVECTOR[i]
        )
    )

### Create Parametrized Function


In [ ]:
UNFOLDED_EXPRESSIONS_FVECTOR = []
INTENSITY_FUNCS_FVECTOR = []
for i in range(n_channels):
    UNFOLDED_EXPRESSIONS_FVECTOR.append(FULL_EXPRESSIONS_FVECTOR[i].doit())
    INTENSITY_FUNCS_FVECTOR.append(
        create_parametrized_function(
            expression=UNFOLDED_EXPRESSIONS_FVECTOR[i],
            backend="jax",
            parameters=MODELS_FVECTOR[i].parameter_defaults,
        )
    )

## Update parameters

In [ ]:
m_res = 1.82
g_res_ch0 = 1.8
g_res_ch1 = 2.5

new_parameters_fvector = {
    R"m_{N^{**}_1}": 1.71,
    R"\beta_{N^{**}_1}": 1 + 0j,
    R"g_{N^{**}_1,0}": g_res_ch0,
    R"g_{N^{**}_1,1}": g_res_ch1,
}

In [ ]:
for i in range(n_channels):
    INTENSITY_FUNCS_FVECTOR[i].update_parameters(new_parameters_fvector)

##  Generate data with $F$ vector

### Generate phase space sample

In [ ]:
HELICITY_TRANSFORMERS = []
for i in range(n_channels):
    HELICITY_TRANSFORMERS.append(
        SympyDataTransformer.from_sympy(
            MODELS_FVECTOR[i].kinematic_variables, backend="jax"
        )
    )

In [ ]:
re.match(r"^m_\d\d$", "m_01")

In [ ]:
PHSP = []
epsilon = 1e-8
for i in range(n_channels):
    rng = TFUniformRealNumberGenerator(seed=0)
    phsp_generator = TFPhaseSpaceGenerator(
        initial_state_mass=REACTIONS[i].initial_state[-1].mass,
        final_state_masses={it: p.mass for it, p in REACTIONS[i].final_state.items()},
    )
    phsp_momenta = phsp_generator.generate(100_000, rng)
    phsp = HELICITY_TRANSFORMERS[i](phsp_momenta)
    phsp = {k: v.real for k, v in phsp.items()}
    phsp = {
        k: v + epsilon * 1j if re.match(r"^m_\d\d$", k) else v for k, v in phsp.items()
    }
    PHSP.append(phsp)

In [ ]:
PHSP[1]

In [ ]:
INTENSITY_FUNCS_FVECTOR[0](PHSP[0])

In [ ]:
INTENSITY_FUNCS_FVECTOR[0].parameters

### Dynamics expressions

In [ ]:
DYNAMICS_EXPR_FVECTOR = []
for i in range(n_channels):
    values, *_ = DYNAMICS_EXPRESSIONS_FVECTOR[i].values()
    DYNAMICS_EXPR_FVECTOR.append(values)

In [ ]:
DYNAMICS_FUNCS_FVECTOR = []
for i in range(n_channels):
    func = create_parametrized_function(
        expression=DYNAMICS_EXPR_FVECTOR[i].doit(),
        backend="jax",
        parameters=MODELS_FVECTOR[i].parameter_defaults,
        use_cse=False,
    )
    DYNAMICS_FUNCS_FVECTOR.append(func)

### Weighted data with $F$ vector 

In [ ]:
for i in range(n_channels):
    fig, ax = plt.subplots(figsize=(6, 5))
    intensity = np.real(INTENSITY_FUNCS_FVECTOR[i](PHSP[i]))
    c = ax.hist(
        np.real(PHSP[i]["m_01"]) ** 2,
        bins=100,
        weights=intensity,
    )
    ax.set_xlabel(R"$M^2\left(\eta p\right)\, \mathrm{[(GeV/c)^2]}$")
    ax.set_ylabel(R"Intensity [a.u.]")
    fig.tight_layout()
    plt.show()

In [ ]:
DATA = []
for i in range(n_channels):
    weighted_phsp_generator = TFWeightedPhaseSpaceGenerator(
        initial_state_mass=MODELS[i].reaction_info.initial_state[-1].mass,
        final_state_masses={
            i: p.mass for i, p in MODELS[i].reaction_info.final_state.items()
        },
    )
    data_generator = IntensityDistributionGenerator(
        domain_generator=weighted_phsp_generator,
        function=INTENSITY_FUNCS_FVECTOR[i],
        domain_transformer=HELICITY_TRANSFORMERS[i],
    )
    data_momenta = data_generator.generate(50_000, rng)
    pd.DataFrame({
        (k, label): np.transpose(v)[i]
        for k, v in data_momenta.items()
        for i, label in enumerate(["E", "px", "py", "pz"])
    })
    phsp = HELICITY_TRANSFORMERS[i](phsp_momenta)
    data = HELICITY_TRANSFORMERS[i](data_momenta)
    DATA.append(data)

In [ ]:
for i in range(n_channels):
    resonances = sorted(
        MODELS[i].reaction_info.get_intermediate_particles(),
        key=lambda p: p.mass,
    )
    evenly_spaced_interval = np.linspace(
        0, 1, len(INTENSITY_FUNCS_FVECTOR[i].parameters.items())
    )
    colors = [cm.rainbow(x) for x in evenly_spaced_interval]
    fig, ax = plt.subplots(figsize=(9, 4))
    ax.hist(
        np.real(DATA[i]["m_01"]),
        bins=200,
        alpha=0.5,
        density=True,
    )
    ax.set_xlabel("$m$ [GeV]")
    for (k, v), color in zip(new_parameters_fvector.items(), colors):
        if k.startswith("m_{"):
            ax.axvline(
                x=v,
                linestyle="dotted",
                label=r"$" + k + "$",
                color=color,
            )
    ax.legend()
    plt.show()

## Perform fit

### Set initial parameters 

In [ ]:
initial_parameters = {
    R"m_{N^{**}_1}": 1.9,
    R"\beta_{N^{**}_1}": 1 + 0j,
    R"g_{N^{**}_1,0}": 2.8,
    R"g_{N^{**}_1,1}": 1.6,
}
INTENSITY_FUNCS_FVECTOR[0].parameters

In [ ]:
def indicate_masses(ax, function):
    ax.set_xlabel("$m$ [GeV]")
    for (k, v), color_F in zip(function.parameters.items(), colors_F):
        if k.startswith("m_{N"):
            ax.axvline(
                x=v,
                linestyle="dotted",
                label=r"$" + k + "$" "(F vector)",
                color=color_F,
            )


def compare_model(
    variable_name: str,
    data: DataSample,
    phsp: DataSample,
    function: Function[DataSample, np.ndarray],
    bins: int = 100,
):
    fig, ax = plt.subplots(figsize=(9, 4))
    ax.hist(
        data[variable_name].real,
        bins=bins,
        alpha=0.5,
        label="data",
        density=True,
    )
    intensities = function(phsp)
    ax.hist(
        phsp[variable_name].real,
        weights=intensities,
        bins=bins,
        histtype="step",
        color="red",
        label="Fit model with $F$ vector",
        density=True,
    )
    indicate_masses(ax, function)
    ax.axvline(
        DECAYS[0].child1.mass + DECAYS[0].child2.mass,
        color="grey",
        linestyle="dotted",
        label=rf"${DECAYS[0].child1.latex} \, {DECAYS[0].child2.latex}$ threshold",
    )
    ax.axvline(
        DECAYS[1].child1.mass + DECAYS[1].child2.mass,
        color="grey",
        linestyle="dotted",
        label=rf"${DECAYS[1].child1.latex} \, {DECAYS[1].child2.latex}$ threshold",
    )
    ax.legend()
    fig.show()

In [ ]:
ORIGINAL_PARAMETERS_F = []
for i in range(n_channels):
    resonances = sorted(
        MODELS[i].reaction_info.get_intermediate_particles(),
        key=lambda p: p.mass,
    )
    evenly_spaced_interval = np.linspace(
        0, 1, len(INTENSITY_FUNCS_FVECTOR[i].parameters.items())
    )
    colors_F = [cm.rainbow(x) for x in evenly_spaced_interval]
    original_parameters = INTENSITY_FUNCS_FVECTOR[i].parameters
    ORIGINAL_PARAMETERS_F.append(original_parameters)
    INTENSITY_FUNCS_FVECTOR[i].update_parameters(initial_parameters)
    compare_model("m_01", DATA[i], PHSP[i], INTENSITY_FUNCS_FVECTOR[i])

### Define estimator

In [ ]:
ESTIMATORS_F = []
for i in range(n_channels):
    estimator_fvector = UnbinnedNLL(
        INTENSITY_FUNCS_FVECTOR[i],
        data=DATA[i],
        phsp=PHSP[i],
        backend="jax",
    )
    ESTIMATORS_F.append(estimator_fvector)

In [ ]:
class EstimatorSum(Estimator):
    def __init__(self, estimators: Iterable[Estimator]) -> None:
        self.__estimators = tuple(estimators)

    def __call__(self, parameters: Mapping[str, ParameterValue]) -> float:
        return sum(estimator(parameters) for estimator in self.__estimators)

    def gradient(
        self, parameters: Mapping[str, ParameterValue]
    ) -> dict[str, ParameterValue]:
        raise NotImplementedError

In [ ]:
combined_estimators = EstimatorSum(ESTIMATORS_F)

## Optimized fit

In [ ]:
minuit2 = Minuit2(
    callback=CSVSummary("fit_traceback.csv"),
    use_analytic_gradient=False,
)
fit_result = minuit2.optimize(combined_estimators, initial_parameters)
fit_result

In [ ]:
for i in range(n_channels):
    INTENSITY_FUNCS_FVECTOR[i].update_parameters(fit_result.parameter_values)
    compare_model("m_01", DATA[i], PHSP[i], INTENSITY_FUNCS_FVECTOR[i])

In [ ]:
original_parameters = {
    **ORIGINAL_PARAMETERS_F[0],
    **ORIGINAL_PARAMETERS_F[1],
}
df = pd.DataFrame({
    f"${p}$": (
        initial_parameters[p],
        fit_result.parameter_values[p],
        original_parameters[p],
    )
    for p in fit_result.parameter_values
}).T
df.columns = ("initial", "fit result", "original")
df.round(decimals=3)

In [ ]:
fit_result

In [ ]:
n_real_par = fit_result.count_number_of_parameters(complex_twice=True)
n_events = len(next(iter(data.values())))
log_likelihood = -fit_result.estimator_value
log_likelihood

In [ ]:
aic = 2 * n_real_par - 2 * log_likelihood
aic

In [ ]:
bic = n_real_par * np.log(n_events) - 2 * log_likelihood
bic